In [1]:
#define quantum circuit to implement query 
from qiskit import QuantumCircuit
def deutsch_function(case: int):
    """ 
    Gnerate a valid Deustch function as 'QuantumCircuit'
    """ 
    if case not in [1,2,3,4]:
        raise Valuerror("'case' must be 1,2,3, or 4.")

    f = QuantumCircuit(2) 
    if case in [2,3]:
        f.cx(0, 1)
    if case in [3,4]:
        f.x(1)
    return f 

In [2]:
deutsch_function(3).draw()

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

In [3]:
def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in Deutsch's algorithm.
    """
    n = function.num_qubits-1
    qc = QuantumCircuit(n+1, n)

    qc.x(n)
    qc.h(range(n+1))

    qc.barrier()
    qc.compose(function, inplace=True)
    qc.barrier()

    qc.h(range(n))
    qc.measure(range(n), range(n))

    return qc

In [4]:
compile_circuit(
    deutsch_function(3)
).draw()

┌───┐      ░            ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■────────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║ 
c: 1/════════════════════════════════╩═
                                     0

In [5]:
from qiskit_aer import AerSimulator 

def deutsch_algorithm(function: QuantumCircuit):
    """
    Determine if a Deutsch function is constant or balanced. 
    """

    qc = compile_circuit(function)

    result = AerSimulator().run(qc,shorts=1,memory=True).result()
    measurement = result.get_memory()
    if measurement[0] == '0':
        return 'constant'
    return 'balanced'

In [6]:
f = deutsch_function(3)
display(f.draw())
deutsch_algorithm(f)


q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

'balanced'

In [7]:
from qiskit import QuantumCircuit
import numpy as np

def dj_function(num_qubits):

    """
    Create a random Deutsch-Jozsa function.
    """

    qc = QuantumCircuit(num_qubits+1)
    if np.random.randint(0, 2):
        #flip output qubit with 50% chance
        qc.x(num_qubits)
    if np.random.randint(0, 2):
        #return constant circuit with 50% chance
        return qc 

    #next, choose half the possible input states
    on_states = np.random.choice(
        range(2**num_qubits), #number to sample from 
        2**num_qubits // 2, #number of samples
        replace=False # makes sure states are only sampled once. 
    )

    def add_cx(qc, bit_string):
        for qubit, bit in enumerate(reversed(bit_string)):
            if bit == '1':
                qc.x(qubit)
        return qc 

    for state in on_states:
        qc.barrier() # Barriers are added to help visualize how the function are created. they can safely be removed. 
        qc = add_cx(qc, f"{state:0b}")
        qc.mct(list(range(num_qubits)), num_qubits)
        qc = add_cx(qc, f"{state:0b}")

    qc.barrier()

    return qc

In [8]:
dj_function(3).draw()

q_0: 
     
q_1: 
     
q_2: 
     
q_3:

In [11]:
#finally a function that runs the Deutsch-Jozsa circuit once is defined

from qiskit_aer import AerSimulator 

def dj_algorithm(function: QuantumCircuit):
    """
    Determined if Deutsch-Josza fucntion is constant or balance. 
    """

    qc = compile_circuit(function)

    result  = AerSimulator().run(qc,shot-1,memory=True).result()
    measurements = result.get_memory()
    if '1' in measurement[0]:
        return 'balanced'
    return 'constant'

In [12]:
def bv_function(s):
    """
    Create a Bernstein-Vazirani function from a string of 1s and 0s.
    """
    qc = QuantumCircuit(len(s)+1)
    for index, bit in enumerate(reversed(s)):
        if bit == '1':
            qc.cx(index, len(s))
    return qc

bv_function('1001').draw()

q_0: ──■───────
       │       
q_1: ──┼───────
       │       
q_2: ──┼───────
       │       
q_3: ──┼────■──
     ┌─┴─┐┌─┴─┐
q_4: ┤ X ├┤ X ├
     └───┘└───┘